In [12]:
# import standard libraries
import glob
import re
# import third-party libraries
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier

# import local libraries
from mlf import TreeClassifier

In [2]:
NUM_TOPICS = glob.glob(r'data/nlp/*.gensim')
NUM_TOPICS = [int(re.findall(r'\d+', s)[0]) for s in NUM_TOPICS]
NUM_DAYS = [2, 3, 5, 7, 14, 21, 28, 35, 42, 49, 56]

In [3]:
clf = TreeClassifier()
_, _ = clf.generate_xy(num_topics=100, num_days=28)
_ = clf.scale_x()

/home/williammartin/miniconda3/envs/epfl/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/home/williammartin/Github/mlf-project/mlf/mlf_utils.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['num_app_prior'] = np.nan


In [4]:
pca = PCA(n_components=0.6, svd_solver='full')
pca = pca.fit(clf.X_scaled)
X = pca.transform(clf.X_scaled)
print(len(pca.explained_variance_ratio_))
pca.explained_variance_ratio_.cumsum()

53


array([0.02456131, 0.04439649, 0.0612942 , 0.07680496, 0.09211194,
       0.10576675, 0.11931718, 0.13266696, 0.14558268, 0.15805411,
       0.17023373, 0.18229309, 0.19427104, 0.20599387, 0.21761897,
       0.22904585, 0.24044846, 0.25178786, 0.26304012, 0.27419757,
       0.28524867, 0.29621897, 0.30715141, 0.31786666, 0.32852073,
       0.33914701, 0.34971641, 0.36021848, 0.37064035, 0.38101714,
       0.39138263, 0.40165432, 0.41189615, 0.42197842, 0.43205486,
       0.44206429, 0.45199882, 0.46188398, 0.47176358, 0.48160484,
       0.49137686, 0.50110173, 0.51079524, 0.52035882, 0.52990834,
       0.53945225, 0.54894108, 0.55842176, 0.56786014, 0.57727324,
       0.58658756, 0.59586894, 0.60510078])

In [6]:
param_grid = {'min_samples_split': [5, 10, 50],
             'criterion': ['gini'],
             'min_samples_leaf': [1, 2, 5, 10],
             'max_leaf_nodes': [None, 10, 20, 50]} 

# separate train and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    clf.y['y_bin'],
                                                    test_size=0.3,
                                                    random_state=0,
                                                    stratify=clf.y['y_bin'])

cv = GridSearchCV(clf, param_grid)
cv = cv.fit(X_train, y_train)
print(cv.score(X_train, y_train))
print(cv.score(X_test, y_test))

0.5713865607998824
0.5584905660377358


In [7]:
opt_est = {k: cv.best_estimator_.__getattribute__(k) for k, _ in param_grid.items()}
opt_est

{'min_samples_split': 5,
 'criterion': 'gini',
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10}

## Try bagging to increase performance

use best estimators found from grid search cross validation

In [40]:
bagger = BaggingClassifier(base_estimator=TreeClassifier(**opt_est), 
                           n_estimators=1000, random_state=0, max_samples=100,
                           warm_start=True)
bagger = bagger.fit(X_train, y_train)

In [41]:
print(bagger.score(X_train, y_train))
print(bagger.score(X_test, y_test))

0.5746213792089399
0.5643224699828473
